In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load the IMDb dataset
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

# Example batch
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

# Define the embedding layer from TensorFlow Hub
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

# Example of how the hub layer processes the data
hub_layer(train_example_batch[:3])

# Build the model
model = tf.keras.Sequential()
model.add(hub_layer)  # This layer transforms the input strings into embeddings
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data.shuffle(10000).batch(100), epochs=25,
                    validation_data=validation_data.batch(100), verbose=1)

# Evaluate the model on the test data
results = model.evaluate(test_data.batch(100), verbose=2)

for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.3f}")


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x0000021D7FE7A7B0> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load the IMDb dataset
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

# Define the embedding layer from TensorFlow Hub
embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding_url, input_shape=[], dtype=tf.string, trainable=True)

# Functional API model definition
inputs = tf.keras.Input(shape=(), dtype=tf.string)
x = hub_layer(inputs)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data.shuffle(10000).batch(100), epochs=25,
                    validation_data=validation_data.batch(100), verbose=1)

# Evaluate the model on the test data
results = model.evaluate(test_data.batch(100), verbose=2)

for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.3f}")



TypeError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

Binding inputs to tf.function failed due to `A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
`. Received args: (<KerasTensor shape=(None,), dtype=string, sparse=None, name=keras_tensor>,) and kwargs: {} for signature: (sentences: TensorSpec(shape=(None,), dtype=tf.string, name=None)).

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load the IMDb dataset
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

# Define the embedding layer from TensorFlow Hub
embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

class CustomHubLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_url):
        super(CustomHubLayer, self).__init__()
        self.hub_layer = hub.KerasLayer(embedding_url, dtype=tf.string, trainable=False)

    def call(self, inputs):
        return self.hub_layer(inputs)

# Use the custom layer
custom_hub_layer = CustomHubLayer(embedding_url)

# Build the model using the Functional API
inputs = tf.keras.Input(shape=(), dtype=tf.string)
x = custom_hub_layer(inputs)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data.shuffle(10000).batch(100),
    epochs=200,
    validation_data=validation_data.batch(100),
    verbose=1
)


# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_data.batch(100))
print(f"\nTest Accuracy: {test_acc:.3f}")


Epoch 1/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.5077 - loss: 0.7933 - val_accuracy: 0.6059 - val_loss: 0.6428
Epoch 2/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.6201 - loss: 0.6305 - val_accuracy: 0.6322 - val_loss: 0.6120
Epoch 3/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.6548 - loss: 0.6013 - val_accuracy: 0.6435 - val_loss: 0.5972
Epoch 4/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - accuracy: 0.6658 - loss: 0.5888 - val_accuracy: 0.6522 - val_loss: 0.5878
Epoch 5/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - accuracy: 0.6679 - loss: 0.5818 - val_accuracy: 0.6786 - val_loss: 0.5792
Epoch 6/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.6780 - loss: 0.5757 - val_accuracy: 0.6732 - val_loss: 0.5753
Epoch 7/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.6820 - loss: 0.5733 - val_accuracy: 0.6919 - val_loss: 0.5722
Epoch 8/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.6869 - loss: 0.5706 - 